In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# uninstall
!pip uninstall -y wandb

# download
!pip install transformers

!pip install --upgrade accelerate

!pip install openai

!pip install python-dotenv

# import
import re
import json
import torch
import random
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import re
import json
import torch
import random
import os
import time
#import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
#from sklearn.metrics import f1_score
#from sklearn.model_selection import train_test_split
#imports OpenAI
import openai
from dotenv import load_dotenv, find_dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.8 MB/s eta 0:00:00


In [5]:
_ = load_dotenv(find_dotenv()) # read local .env file

#openai.api_key = os.environ["OPENAI_API_KEY"]

#configure aqui sua apikey da openai
openai.api_key = 'SUA_API_KEY'

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def predict(text):
    prompt_text = f"""
        return 'negative' if the sentiment is negative or return 'positive' \
        if the sentiment is positive for the following text: '''{text}'''
        """
    response = get_completion(prompt_text)

    return response

    #prompt_text = f"""
    #Please keep {response} JSON format, but translate keys and values to portuguese from brazil.

    #No explanation is needed.
    #"""
    #response = get_completion(prompt_text)

In [6]:
print(predict(str("Gostaria de saber o que houve cm a questão de conseguir olhar o caminho que o motoboy está percorrendo quando se inicia a nossa entrega!? Antes ficava mais fácil aguardar, pq víamos quando estava chegando, agora não aparece mais. É algum bug ou fizeram a retirada?").lower()))

negative


In [7]:
# Dataset class
class SentimentDataset(Dataset):
    def __init__(self, txt_list, label_list, tokenizer, max_length):
        # define variables
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        map_label = {0:'negative', 4: 'positive'}
        # iterate through the dataset
        for txt, label in zip(txt_list, label_list):
            # prepare the text
            prep_txt = f'<|startoftext|>Review: {txt}\nSentiment: {map_label[label]}<|endoftext|>'
            # tokenize
            encodings_dict = tokenizer(prep_txt, truncation=True,
                                       max_length=max_length, padding="max_length")
            # append to list
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
            self.labels.append(map_label[label])

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]

# Data load function
def load_sentiment_dataset(tokenizer, random_seed = 1, file_path="/content/drive/My Drive/Colab/reviews_final.csv"):
    # load dataset and sample 10k reviews.
    #df = pd.read_csv(file_path, encoding='ISO-8859-1', header=None)
    #df = pd.read_csv(file_path, header=True, on_bad_lines='skip')
    df = pd.read_csv(file_path, header=None, on_bad_lines='skip')
    #df = df[[0, 5]]
    df = df[[0, 1]]
    df.columns = ['label', 'text']
    #df = df.fillna(0)
    #df['label'] = df['label'].astype(int)
    df = df.sample(7400, random_state=0)
    #df = df[df['text'].notna()]

    ##DEBUG
    #pd.set_option('display.max_rows', None)
    #print(df)



    def pick_first_n_words(string, max_words=250): # tried a few max_words, kept 250 as max tokens was < 512
        #print(string)
        split_str = string.split()
        #print(split_str[:min(len(split_str), max_words)])
        return " ".join(split_str[:min(len(split_str), max_words)])

    df['text'] = df['text'].apply(lambda x: pick_first_n_words(x))

    print("############")


    # divide into test and train
    X_train, X_test, y_train, y_test = \
              train_test_split(df['text'].tolist(), df['label'].tolist(),
              shuffle=True, test_size=0.05, random_state=random_seed, stratify=df['label'])

    # get max length
    max_length_train = max([len(tokenizer.encode(text)) for text in X_train])
    max_length_test = max([len(tokenizer.encode(text)) for text in X_test])
    max_length = max([max_length_train, max_length_test]) + 10  #for special tokens (sos and eos) and fillers
    max_length = max(max_length, 300)
    print(f"Setting max length as {max_length}")

    # format into SentimentDataset class
    train_dataset = SentimentDataset(X_train, y_train, tokenizer, max_length=max_length)

    # return
    return train_dataset, (X_test, y_test)

In [8]:
# import
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

# model
model_name = "gpt2"
seed = 42

# seed
torch.manual_seed(seed)

In [9]:
# iterate for N trials
for trial_no in range(3):

  print("Loading model...")
  # load tokenizer and model
  tokenizer = GPT2Tokenizer.from_pretrained(model_name, bos_token='<|startoftext|>',
                                                eos_token='<|endoftext|>', pad_token='<|pad|>')
  model = GPT2LMHeadModel.from_pretrained(model_name).cuda()
  model.resize_token_embeddings(len(tokenizer))

  print("Loading dataset...")
  train_dataset, test_dataset = load_sentiment_dataset(tokenizer, trial_no)

    #print("Start training...")
    #training_args = TrainingArguments(output_dir='results', num_train_epochs=10,
                                    #logging_steps=10, load_best_model_at_end=True,
                                     # save_strategy="no", evaluation_strategy="no", per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                    #warmup_steps=100, weight_decay=0.01, logging_dir='logs')

    #Trainer(model=model, args=training_args, train_dataset=train_dataset,
            #eval_dataset=test_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                                  #'attention_mask': torch.stack([f[1] for f in data]),
                                                                  #'labels': torch.stack([f[0] for f in data])}).train()

    # test
    #print("Start testing...")
    # eval mode on model
    #_ = model.eval()

Loading model...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading dataset...
############
Setting max length as 355
Loading model...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading dataset...
############
Setting max length as 355
Loading model...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading dataset...
############
Setting max length as 355


In [10]:
#Realizando consultas ao modelo treinado da OpenAI:
predicted2 = []
for x in range(370):
  try:
    res = predict(str(test_dataset[0][x]).lower())
  except:
    print("skipping at " + x)
  predicted2.insert(x, res)
  #predicted2.append(res)
  time.sleep(1)
  print(x)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
print(test_dataset[0][369])
print(predicted2[369])


Muito mais intuitivo que os mais famosos; bem brasileiro/ carioca no linguajar. Gosto muito.
positive


In [11]:
#Calculando a diferença entre o test_dataset e o predict_gpt3:
i = 0
for x in range(370):
  #print(x)
  if (test_dataset[1][x] == 4 and predicted2[x] != 'positive') or \
   (test_dataset[1][x] == 0 and predicted2[x] != 'negative'):
    #print(test_dataset[0][x])
    i = i + 1
    #print(x)

aproveitamento = 100 - ((100*i)/370)
print("\n\nHouve " + str(i) + " difenças..\nEquivalente a um aproveitamento\
 de " + str(aproveitamento) + "%")



Houveram 44 difenças..
Equivalente a um aproveitamento de 88.10810810810811%


In [ ]:
# compute prediction on test data
original, predicted, all_text, predicted_text = [], [], [], []
map_label = {0:'negative', 4: 'positive'}
for text, label in tqdm(zip(test_dataset[0], test_dataset[1])):
        # predict sentiment on test data
        prompt = f'<|startoftext|>Review: {text}\nSentiment:'
        generated = tokenizer(f"<|startoftext|> {prompt}", return_tensors="pt").input_ids.cuda()
        sample_outputs = model.generate(generated, do_sample=False, top_k=50, max_length=512, top_p=0.90,
                temperature=0, num_return_sequences=0, pad_token_id=tokenizer.eos_token_id)
        pred_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
        # extract the predicted sentiment
        try:
            #pred_sentiment = re.findall("\nSentiment: (.*)", pred_text)[-1]
            pred_sentiment = predict(str(pred_text).lower())
            print(pred_sentiment)
        except:
            pred_sentiment = "None"
        original.append(map_label[label])
        predicted.append(pred_sentiment)
        all_text.append(text)
        predicted_text.append(pred_text)
#transform into dataframe
df = pd.DataFrame({'text': all_text, 'predicted': predicted, 'original': original, 'predicted_text': predicted_text})
df.to_csv(f"result_run_{trial_no}.csv", index=False)
# compute f1 score
print(f1_score(original, predicted, average='macro'))

1it [00:07,  7.24s/it]

negative


2it [00:14,  7.06s/it]

negative


3it [00:21,  7.05s/it]

...

negative


369it [1:06:06,  7.39s/it]

negative


370it [1:06:14, 10.74s/it]

positive
0.433962543708884
